In [1]:
# designed for assessing which games are the most popular, then exports the best to a json file
# so steam api data can be restricted

In [1]:
import pandas as pd
import keyring
from queryer import QueryApi
from ast import literal_eval
import json
from IPython.display import clear_output

In [2]:
password = keyring.get_password(service_name = 'steam', username = 'IcyJoseph')

In [3]:
api = QueryApi(password)

Successfully connected to the Steam API


In [13]:
df = pd.read_csv('popular_games.csv', usecols = ['players_estimate', 'steam_app_id'])
df = df.set_index('steam_app_id')

In [37]:
df.head()

,players_estimate
steam_app_id,
440,"50,191,347"
730,"46,305,966"
578080,"36,604,134"
304930,"27,381,399"
550,"23,143,723"


In [42]:
df = df.sort_values('players_estimate',ascending = False)
top_games = list(df.index[:1000])

In [33]:
temp_df = pd.DataFrame(index = [0], data = {'app_id':'440', 'game_name':api.get_game_info(game)['name']})

In [14]:
game_lookup = pd.DataFrame(columns = ['app_id','game_name','genre'])

for game in list(df.index)[:5]:
    
    try:
        gamereturn = api.get_game_info(game)
        temp_df = pd.DataFrame(index = [0], 
                data = {'app_id':game, 'game_name':gamereturn['name'], 'genre':gamereturn['genre']})
        game_lookup = game_lookup.append(temp_df)
    
    except json.decoder.JSONDecodeError:
        continue
    
    clear_output()
    print(list(df.index).index(game))
    
game_lookup = game_lookup.set_index('app_id')

4


In [50]:
game_lookup.head()

,game_name
app_id,
258970,Gauntlet Slayer Edition
367520,Hollow Knight
261570,Ori and the Blind Forest
412530,Atlantic Quest 2 - New Adventure -
365340,The Way of the Pixelated Fist
